In [303]:
import pandas as pd

## Task #1
С помощью подхода **MapReduce** посчитайте **среднюю погоду по каждому городу за представленный период**

In [304]:
#Reading data

weather = pd.read_csv('city_weather.csv', sep=',')

### t1 / mapping
1. Предположим, что для каждого города существует индекс.
2. Для всех городов, кроме Грозного, Владикавказа и Челябинска, количество измерений равно 31. Для того чтобы не считать количество дней для каждого города на кластере в цикле добавим столбец с этой информацией на этапе маппинга. 

In [305]:
sort_unq_city_lst = weather.city.unique()
sort_unq_city_lst.sort()

In [306]:
sort_unq_city_lst

array(['Абакан', 'Анадырь', 'Архангельск', 'Астрахань', 'Барнаул',
       'Биробиджан', 'Благовещенск', 'Брянск', 'Великий Новгород',
       'Владивосток', 'Владикавказ', 'Владимир', 'Волгоград', 'Вологда',
       'Воронеж', 'Грозный', 'Екатеринбург', 'Иваново', 'Ижевск',
       'Иркутск', 'Йошкар-Ола', 'Казань', 'Калининград', 'Калуга',
       'Кемерово', 'Киров', 'Кострома', 'Краснодар', 'Курган', 'Курск',
       'Кызыл', 'Липецк', 'Магадан', 'Майкоп', 'Махачкала', 'Москва',
       'Мурманск', 'Нальчик', 'Нижний Новгород', 'Новосибирск', 'Омск',
       'Орел', 'Оренбург', 'Пенза', 'Пермь', 'Петрозаводск',
       'Петропавловск-Камчатский', 'Псков', 'Ростов', 'Рязань',
       'Салехард', 'Самара', 'Санкт-Петербург', 'Саранск', 'Смоленск',
       'Ставрополь', 'Сыктывкар', 'Тамбов', 'Тверь', 'Томск', 'Тула',
       'Тюмень', 'Улан-Удэ', 'Уфа', 'Хабаровск', 'Ханты-Мансийск',
       'Чебоксары', 'Челябинск', 'Чита', 'Элиста', 'Южно-Сахалинск',
       'Якутск'], dtype=object)

In [307]:
# Добавляем индекс для каждого города
sort_unq_city_lst = weather.city.unique()
sort_unq_city_lst.sort()

city_to_idx = dict(zip(sort_unq_city_lst,range(1, 73)))

In [308]:
# Добавим столбец с количеством измерений для каждого города

with open('data_t1/city_weather_after_mapper.csv', 'w') as f_mapper:
    with open('city_weather.csv') as f:
        for i, line in enumerate(f):
            if not i:
                continue
            
            city, date, day_temp, night_temp = line.strip().split(',')
    
            idx_city = city_to_idx[city]
            if city == 'Грозный' or city == 'Владикавказ':
                periods_num = 5
            elif city == 'Челябинск':
                periods_num = 17
            else:
                periods_num = 31
            f_mapper.write(f'{idx_city}, {day_temp}, {night_temp}, {periods_num}\n')
            

### t1 / shuffle
Сортируем полученный после маппинга датафрейм по индексу города

In [309]:
! cat data_t1/city_weather_after_mapper.csv | sort -k1,1n > data_t1/city_weather_am_sorted.csv

### t1 / reduce

In [310]:
with open('data_t1/city_weather_reducer.csv', 'w') as f_reducer:
    f_reducer.write('idx_city,avg_dtemp,avg_ntemp,avg_temp\n')
    with open('data_t1/city_weather_am_sorted.csv') as f:
        day_num, ttl_dtemp, ttl_ntemp = 1, 0, 0
        for i, line in enumerate(f):
            idx_city, day_temp, night_temp, periods_num = line.strip().split(',')
            periods_num = int(periods_num)
            ttl_dtemp += float(day_temp)
            ttl_ntemp += float(night_temp)

            if day_num == periods_num:
                f_reducer.write(
                    f'{idx_city}, \
                    {round(ttl_dtemp / periods_num, 2)}, \
                    {round(ttl_ntemp / periods_num, 2)}, \
                    {round((ttl_dtemp + ttl_ntemp) / periods_num / 2, 2)}\n')
                day_num, ttl_dtemp, ttl_ntemp = 1, 0, 0
            else:
                day_num += 1

### Ответ на task #1
После вычислений получаем следующий датафрейм: \
idx_city - индекс города \
avg_dtemp - средняя температура в городе в дневное время суток \
avg_ntemp - средняя температура в городе в ночное время суток \
avg_temp - средняя температура в городе

In [311]:
pd.read_csv('data_t1/city_weather_reducer.csv', sep=',').head()

,idx_city,avg_dtemp,avg_ntemp,avg_temp
0,1,3.87,-7.16,-1.65
1,2,-7.74,-17.52,-12.63
2,3,1.72,-7.23,-2.75
3,4,13.94,1.71,7.82
4,5,1.21,-8.86,-3.83


## Task #2
С помощью подхода **MapReduce** посчитайте **среднюю температуру по федеральному округу**

В качестве данных с температурой по городам будем использовать датафрейм из прошлой задачи city_weather_am_sorted.csv

### t2 / mapping
0. Чистим district_to_city.csv от дублей в атрибуте city
1. Как и в первой задаче, предположим, что для каждого города существует индекс, а также для каждого региона.
2. Для всех городов, кроме Грозного, Владикавказа и Челябинска, количество измерений равно 31. Добавим столбец с периодом измерений на этапе маппинга для каждого региона.
3. Добавим дополнительные столбцы для конкатенации датафреймов.

In [292]:
# Regions pre-mapping. Adding quantity of measurements for each region
regions = pd.read_csv('district_to_city.csv', sep=',').query('city in @sort_unq_city_lst').drop_duplicates(subset=['city'])

regions['periods_num'] = [31 if city not in ['Грозный', 'Владикавказ', 'Челябинск'] else 5 if city in ['Грозный', 'Владикавказ'] else 17 if city == 'Челябинск' else 0 for city in regions.city]
regions['region_periods_num'] = regions.groupby('federal_district')['periods_num'].transform('sum')
regions[['federal_district', 'city', 'region_periods_num']].to_csv('data_t2/regions_pre_mapper.csv', sep=',', index=0)

In [294]:
with open('data_t2/t2_after_mapper.csv', 'w') as f_mapper:
    with open('data_t2/regions_pre_mapper.csv') as f:
        regions_to_idx = dict(zip(regions.federal_district.unique(),range(1, 9)))
        city_to_idx = dict(zip(sort_unq_city_lst,range(1, 73)))
        for i, line in enumerate(f):
            if not i:
                continue
            region, city, region_periods_num = line.strip().split(',')
            
            idx_region = regions_to_idx[region]
            idx_city = city_to_idx[city]
            f_mapper.write(f'{idx_region},{idx_city},-,-,{region_periods_num}\n')

    with open('data_t1/city_weather_am_sorted.csv') as f:
        for i, line in enumerate(f):
            idx_city, day_temp, night_temp, periods_num = line.strip().split(',')
            f_mapper.write(f'-,{idx_city},{day_temp},{night_temp},-\n')

In [295]:
! cat data_t2/t2_after_mapper.csv | sort -t, -k2,2n -k1,1nr > data_t2/t2_am_sorted.csv

In [296]:
# Merging
with open('data_t2/t2_pre_reduce.csv', 'w') as f_mapper:
    with open('data_t2/t2_am_sorted.csv') as f:
        for i, line in enumerate(f):
            idx_region, idx_city, day_temp, night_temp, periods_num = line.strip().split(',')
            
            if idx_region != '-':
                curr_region, curr_pnum = idx_region, periods_num
            else:
                f_mapper.write(f'{curr_region},{day_temp},{night_temp},{curr_pnum}\n')

### t2 / shuffle
Сортируем полученный после маппинга датафрейм по индексу региона

In [297]:
! cat data_t2/t2_pre_reduce.csv | sort -t, -k1,1n > data_t2/t2_pre_reduce_sorted.csv

### t2 / reduce

In [313]:
with open('data_t2/t2_reduce.csv', 'w') as f_reducer:
    f_reducer.write('idx_region,avg_dtemp,avg_ntemp,avg_temp\n')
    with open('data_t2/t2_pre_reduce_sorted.csv') as f:
        ttl_dtemp, ttl_ntemp = 0, 0
        fst_line = f.readline().strip().split(',')
        curr_region, curr_pnum = fst_line[0], fst_line[-1]
        for i, line in enumerate(f):
            idx_region, day_temp, night_temp, periods_num = line.strip().split(',')
            if idx_region != curr_region:
                curr_pnum = int(curr_pnum)
                f_reducer.write(
                    f'{curr_region}, \
                    {round(ttl_dtemp / curr_pnum, 2)}, \
                    {round(ttl_ntemp / curr_pnum, 2)}, \
                    {round((ttl_dtemp + ttl_ntemp) / curr_pnum / 2, 2)}\n'
                )
                ttl_dtemp, ttl_ntemp = 0, 0
                curr_region = idx_region
                curr_pnum = periods_num
            ttl_dtemp += float(day_temp)
            ttl_ntemp += float(night_temp)
        curr_pnum = int(curr_pnum)
        f_reducer.write(
                    f'{curr_region}, \
                    {round(ttl_dtemp / curr_pnum, 2)}, \
                    {round(ttl_ntemp / curr_pnum, 2)}, \
                    {round((ttl_dtemp + ttl_ntemp) / curr_pnum / 2, 2)}\n'
                )

### Ответ на task #2
После вычислений получаем следующий датафрейм: \
idx_region - индекс региона \
avg_dtemp - средняя температура в городе в дневное время суток \
avg_ntemp - средняя температура в городе в ночное время суток \
avg_temp - средняя температура в городе

In [316]:
pd.read_csv('data_t2/t2_reduce.csv', sep=',')

,idx_region,avg_dtemp,avg_ntemp,avg_temp
0,1,7.93,-0.53,3.70
1,2,3.49,-5.11,-0.81
2,3,5.45,-2.40,1.52
3,4,3.59,-4.52,-0.47
4,5,1.43,-9.96,-4.26
5,6,-0.75,-11.99,-6.37
6,7,12.59,1.13,6.86
7,8,11.53,2.78,7.15


## Task #3
С помощью подхода **MapReduce** посчитайте **разницу между средней температурой города и федерального округа, к которому относится этот город.**

1. На этапе маппинга нужно получить данные о соответствии названия городов и индексов.
2. Заджойнить три таблицы: датафрейм индексов регионов и городов, датафрейм с температурой по регионам и датафрейм с температурой по городам
3. Посчить разницу между средней температурой города и федерального округа, к которому относится этот город

In [326]:
with open('data_t3/t3_mapper_region_part.csv', 'w') as f_mapper:
    with open('data_t2/regions_pre_mapper.csv') as f:
        regions_to_idx = dict(zip(regions.federal_district.unique(),range(1, 9)))
        city_to_idx = dict(zip(sort_unq_city_lst,range(1, 73)))
        for i, line in enumerate(f):
            if not i:
                continue
            region, city, region_periods_num = line.strip().split(',')
            
            idx_region = regions_to_idx[region]
            idx_city = city_to_idx[city]
            f_mapper.write(f'{idx_region},{idx_city},-,-,-,-,-,-,\n')

    with open('data_t2/t2_reduce.csv') as f:
        for i, line in enumerate(f):
            if not i:
                continue
            idx_region, avg_dtemp, avg_ntemp, avg_temp = line.strip().split(',')
            f_mapper.write(f'{idx_region},-,{avg_dtemp},{avg_ntemp},{avg_temp},-,-,-\n')

In [327]:
! cat data_t3/t3_mapper_region_part.csv | sort -t, -k1,1n -k2,2n > data_t3/t3_am_sorted.csv

In [336]:
# Merging Part1
with open('data_t3/t3_mapper_region_part_clean.csv', 'w') as f_mapper:
    with open('data_t3/t3_am_sorted.csv') as f:
        for i, line in enumerate(f):
            idx_region, idx_city, r_avg_dtemp, r_avg_ntemp, r_avg_temp = line.strip().split(',')[:5]
            
            if idx_city == '-':
                cr_avg_dtemp, cr_avg_ntemp, cr_avg_temp = r_avg_dtemp, r_avg_ntemp, r_avg_temp
            else:
                f_mapper.write(f'{idx_region},{idx_city},{cr_avg_dtemp},{cr_avg_ntemp},{cr_avg_temp},-,-,-\n')

# Merging Part2
with open('data_t3/t3_mapper_city_part.csv', 'w') as f_mapper:
    with open('data_t3/t3_mapper_region_part_clean.csv') as f:
        for i, line in enumerate(f):
            idx_region, idx_city, r_avg_dtemp, r_avg_ntemp, r_avg_temp = line.strip().split(',')[:5]
            f_mapper.write(f'{idx_region},{idx_city},{r_avg_dtemp},{r_avg_ntemp},{r_avg_temp},-,-,-\n')
    
    with open('data_t1/city_weather_reducer.csv') as f:
        for i, line in enumerate(f):
            if not i:
                continue
            idx_city, avg_dtemp, avg_ntemp, avg_temp = line.strip().split(',')
            f_mapper.write(f'-,{idx_city},-,-,-,{avg_dtemp},{avg_ntemp},{avg_temp}\n')

In [337]:
! cat data_t3/t3_mapper_city_part.csv | sort -t, -k2,2n -k1,1n > data_t3/t3_am_sorted_city.csv

In [342]:
with open('data_t3/t3_mapper_clean.csv', 'w') as f_mapper:
    with open('data_t3/t3_am_sorted_city.csv') as f:
        for i, line in enumerate(f):
            idx_region, idx_city, r_avg_dtemp, r_avg_ntemp, r_avg_temp, d_avg_dtemp, d_avg_ntemp, d_avg_temp = line.strip().split(',')
            
            if idx_region == '-':
                cd_avg_dtemp, cd_avg_ntemp, cd_avg_temp = d_avg_dtemp, d_avg_ntemp, d_avg_temp
            else:
                f_mapper.write(f'{idx_region},{idx_city},{r_avg_dtemp},{r_avg_ntemp},{r_avg_temp},{cd_avg_dtemp},{cd_avg_ntemp},{cd_avg_temp}\n')
                

In [349]:
with open('data_t3/t3_reducer.csv', 'w') as f_reducer:
    f_reducer.write('idx_region,idx_city,diff_dtemp,diff_ntemp,diff_temp\n')
    with open('data_t3/t3_mapper_clean.csv') as f:
        for i, line in enumerate(f):
            idx_region, idx_city, r_avg_dtemp, r_avg_ntemp, r_avg_temp, d_avg_dtemp, d_avg_ntemp, d_avg_temp = line.strip().split(',')
            
            f_reducer.write(
                f'{idx_region}, \
                {idx_city}, \
                {round(float(r_avg_dtemp) - float(d_avg_dtemp), 2)}, \
                {round(float(r_avg_ntemp) - float(d_avg_ntemp), 2)}, \
                {round(float(r_avg_temp) - float(d_avg_temp), 2)}, \n')

### Ответ на task #3
После вычислений получаем следующий датафрейм: \
idx_region - индекс региона \
idx_region - индекс города \
diff_dtemp - разница между средней температурой в дневное время суток \
diff_ntemp - разница между средней температурой в ночное время суток \
diff_temp - разница между средней температурой

In [359]:
pd.read_csv('data_t3/t3_reducer.csv', sep=',', index_col=False)

/var/folders/8x/py78bv316g35sfyx4zxjd9vm0000gn/T/ipykernel_91482/1908477133.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  pd.read_csv('data_t3/t3_reducer.csv', sep=',', index_col=False)


,idx_region,idx_city,diff_dtemp,diff_ntemp,diff_temp
0,5,1,-2.44,-2.80,-2.61
1,6,2,6.99,5.53,6.26
2,2,3,1.77,2.12,1.94
3,7,4,-1.35,-0.58,-0.96
4,5,5,0.22,-1.10,-0.43
...,...,...,...,...,...
67,4,68,-0.54,-2.06,-1.31
68,5,69,-0.85,4.35,1.75
69,7,70,-1.00,0.32,-0.34
70,6,71,-2.58,-2.93,-2.75
